<img src='https://raw.githubusercontent.com/autonomio/talos/master/logo.png' width=300px>

## This notebook is an adaptation of the Breath Cancer Example with Functional Model

## Overview

There are four steps to setting up an experiment with Talos:

1) Imports and data

2) Creating the Keras model

3) Defining the Parameter Space Boundaries 

4) Running the Experiment

## 1. The Required Inputs and Data

In [1]:
import talos as ta
import wrangle as wr
#from talos.metrics.keras_metrics import fmeasure

import pandas as pd

# this a locally modified version of the actual package
#from livelossplot import PlotLossesKeras

from keras.models import Sequential, Model
from keras.layers import Dropout, Dense, Input

# Keras items
from keras.optimizers import Adam, Nadam
from keras.activations import relu, elu, sigmoid
from keras.losses import binary_crossentropy

%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# then we load the dataset
x, y = ta.datasets.breast_cancer()

# and normalize every feature to mean 0, std 1
x = wr.mean_zero(pd.DataFrame(x)).values

## 2. Creating the Keras Model

In [3]:
# first we have to make sure to input data and params into the function
def breast_cancer_model(x_train, y_train, x_val, y_val, params):
    
    inputs = Input(shape=(x_train.shape[1],))
    
    layer = Dense(params['first_neuron'], activation=params['activation'], 
                  kernel_initializer=params['kernel_initializer'])(inputs)
    
    layer = Dropout(params['dropout'])(layer)
    
    outputs = Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer'])(layer)
        
    model = Model(inputs, outputs)
    
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](),
                  metrics=['acc'])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0)

    return history, model

## 3. Defining the Parameter Space Boundary

In [4]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[9,10,11],
     'hidden_layers':[0, 1, 2],
     'batch_size': [30],
     'epochs': [100],
     'dropout': [0],
     'kernel_initializer': ['uniform','normal'],
     'optimizer': [Nadam, Adam],
     'losses': [binary_crossentropy],
     'activation':[relu, elu],
     'last_activation': [sigmoid]}

## 4. Starting the Experiment

In [5]:
# and run the experiment
t = ta.Scan(x=x,
            y=y,
            model=breast_cancer_model,
            grid_downsample=1, 
            params=p,
            dataset_name='breast_cancer',
            experiment_no='f',
            experimental_functional_support=True)

100%|██████████| 72/72 [05:48<00:00,  6.39s/it]

Scan Finished!


In [6]:
r = ta.Reporting("breast_cancer_f.csv")

,batch_size,epochs,dropout,kernel_initializer,optimizer,losses,activation,last_activation
val_acc,,,,,,,,
0.994,30,100,0,uniform,Adam,binary_crossentropy,relu,sigmoid
0.994,30,100,0,normal,Adam,binary_crossentropy,relu,sigmoid
0.994,30,100,0,normal,Nadam,binary_crossentropy,relu,sigmoid
0.994,30,100,0,uniform,Nadam,binary_crossentropy,relu,sigmoid
0.994,30,100,0,normal,Nadam,binary_crossentropy,elu,sigmoid
0.994,30,100,0,uniform,Nadam,binary_crossentropy,relu,sigmoid
0.988,30,100,0,uniform,Nadam,binary_crossentropy,relu,sigmoid
0.988,30,100,0,uniform,Adam,binary_crossentropy,relu,sigmoid
0.988,30,100,0,uniform,Nadam,binary_crossentropy,relu,sigmoid


,batch_size,epochs,dropout,kernel_initializer,optimizer,losses,activation,last_activation
val_acc,,,,,,,,
0.988,30,100,0,normal,Adam,binary_crossentropy,relu,sigmoid
0.988,30,100,0,uniform,Adam,binary_crossentropy,elu,sigmoid
0.988,30,100,0,uniform,Nadam,binary_crossentropy,elu,sigmoid
0.988,30,100,0,uniform,Nadam,binary_crossentropy,elu,sigmoid
0.988,30,100,0,uniform,Nadam,binary_crossentropy,relu,sigmoid
0.988,30,100,0,uniform,Adam,binary_crossentropy,relu,sigmoid
0.988,30,100,0,uniform,Nadam,binary_crossentropy,relu,sigmoid
0.988,30,100,0,normal,Adam,binary_crossentropy,elu,sigmoid
0.988,30,100,0,normal,Nadam,binary_crossentropy,relu,sigmoid



 NOTE: you have more options in the Reporting object.

